<a href="https://colab.research.google.com/github/codewithAshwani/Speech-Recognition-CTC-decoder/blob/main/speech_RNN_CTC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import numpy as np
import matplotlib.pyplot as plt
import time
import numpy
from pickle import dump
from pickle import load
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Activation, Bidirectional, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D,TimeDistributed, LSTM

In [ ]:
pickle_path = 'feature.pkl'

In [ ]:
# load data from data file
def load_features(filename):
    data = load(open(filename, 'rb'))

    x = data["feauture_mfcc"]
    y = data["mapping"]
    return x,y

In [ ]:
x,y = load_features(pickle_path)

In [ ]:
max_length = max([len(label) for label in y])
max_length

5

In [ ]:
# padding of words which are smaller than max_length, with character '<unk>'
y_padded = []
for i in range(len(y)):
  val = list(y[i])
  while len(val)<(max_length):
    val.append('<unk>')
  y_padded.append(val)
# characters in label
characters = set(char for label in y for char in label)
# dictionary for tokenization of charaters 
char_to_num = dict(zip(sorted(characters) + ['<unk>'], list(range(len(characters) + 1))))
num_to_char = {v:k for k,v in char_to_num.items()}
# convert characters to integer token 
y2=[]
for label in y_padded:
  y2.append([char_to_num[i] for i in label])

In [ ]:
len(characters)

15

In [ ]:
''.join([num_to_char[i] for i in y2[-1]])

'zero<unk>'

In [ ]:
x = np.asarray(x)
x = x.reshape(-1,x.shape[1],x.shape[2],1)
width = x.shape[1]
height = x.shape[2]
width,height

(55, 13)

In [ ]:
# Split the data
x_train, x_valid, y_train, y_valid = train_test_split(x, np.asarray(y2), test_size=0.01, shuffle= True)

In [ ]:
x_train.shape, y_train.shape, x_valid.shape

((2970, 55, 13, 1), (2970, 5), (30, 55, 13, 1))

In [ ]:
BUFFER_SIZE = x_train.shape[0]
BATCH_SIZE = 16
steps_per_epoch = x_train.shape[0]//BATCH_SIZE
units = 128
dropout = 0.0
beam_widt = 3

dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
BUFFER_SIZE_TEST = x_valid.shape[0]
dataset_test = tf.data.Dataset.from_tensor_slices((x_valid, y_valid)).shuffle(BUFFER_SIZE_TEST)
dataset_test = dataset_test.batch(1)

# Encoder

In [ ]:
class encoder(tf.keras.Model):
    def __init__(self, units, batch_size, characters, dropout, img_width, img_height):
        super(encoder, self).__init__()
        self.batch_size = batch_size
        self.units = units
        self.characters = characters
        self.dropout = dropout
        self.img_width = img_width
        self.img_height = img_height

        self.Conv2D1 = Conv2D(32,(3, 3), activation="relu", kernel_initializer="he_normal", padding="same", name="Conv1")
        self.MaxPooling2D1 = MaxPooling2D((2, 2), name="pool1")
        self.Conv2D2 = Conv2D(64,(3, 3), activation="relu", kernel_initializer="he_normal", padding="same", name="Conv2")
        self.MaxPooling2D2 = MaxPooling2D((2, 2), name="pool2")

        self.Reshape= Reshape(target_shape=((self.img_width // 4), (self.img_height // 4) * 64), name="reshape")
        self.Dense=Dense(64, activation="relu", name="dense1")
        self.BidiLSTM1=Bidirectional(LSTM(units, return_sequences=True, dropout=self.dropout))
        self.BidiLSTM2=Bidirectional(LSTM(units, return_sequences=True, dropout=self.dropout))
        self.Dense1 = TimeDistributed(Dense(len(self.characters)+1, name="dense2"))
        self.Dense2 = TimeDistributed(Activation('softmax'), name='softmax')
    def call(self, x):
        x = self.Conv2D1(x)
        x = self.MaxPooling2D1(x)
        x = self.Conv2D2(x)
        x = self.MaxPooling2D2(x)
        x = self.Reshape(x)
        x = self.Dense(x)
        x = self.BidiLSTM1(x)
        x = self.BidiLSTM2(x)
        x = self.Dense1(x)
        output = self.Dense2(x)
        return output

In [ ]:
encoder = encoder(units, BATCH_SIZE, characters, dropout, width, height)

**Optimizer**

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

class CTCLayer(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
        self.loss_func = tf.keras.backend.ctc_batch_cost

    def call(self, y_true, y_pred):

        batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
        input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
        label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")
        label_length = tf.cast(tf.fill([batch_len, 1], label_length), dtype="int64")
        input_length = tf.cast(tf.fill([batch_len, 1], input_length), dtype="int64")



        loss = self.loss_func(y_true, y_pred, input_length, label_length)
        self.add_loss(loss)
        return y_pred

In [ ]:
CTC_Layer = CTCLayer()

In [ ]:
@tf.function
def train_step(x_batch_train, y_batch_train):
    loss = 0
    with tf.GradientTape() as tape:
        output = encoder(x_batch_train) 
        output = CTC_Layer(y_batch_train, output)
        loss += tf.math.reduce_mean(CTC_Layer.losses)

    batch_loss = (loss / y_batch_train.shape[1])   
    variables = encoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

In [ ]:
EPOCHS = 10
for epoch in range(EPOCHS):
    start = time.time()
    total_loss = 0
    for step, (x_batch_train, y_batch_train) in enumerate(dataset):
        batch_loss = train_step(x_batch_train, y_batch_train)
        total_loss += batch_loss
    print(f'Epoch {epoch+1} Loss {total_loss:.4f}')
    print(f'Time taken for 1 epoch {time.time()-start:.2f} sec\n')

Epoch 1 Loss 415.7357
Time taken for 1 epoch 9.26 sec

Epoch 2 Loss 260.5452
Time taken for 1 epoch 3.21 sec

Epoch 3 Loss 153.4986
Time taken for 1 epoch 3.25 sec

Epoch 4 Loss 95.5731
Time taken for 1 epoch 3.28 sec

Epoch 5 Loss 64.1474
Time taken for 1 epoch 3.19 sec

Epoch 6 Loss 44.8002
Time taken for 1 epoch 3.09 sec

Epoch 7 Loss 33.5921
Time taken for 1 epoch 3.09 sec

Epoch 8 Loss 24.6286
Time taken for 1 epoch 3.07 sec

Epoch 9 Loss 20.2038
Time taken for 1 epoch 3.07 sec

Epoch 10 Loss 17.0169
Time taken for 1 epoch 3.08 sec



In [ ]:
def greedy_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    # Use greedy search.
    results = tf.keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0][
        :, :max_length
    ]
    # Iterate over the results and get back the text
    resultn = np.array([i for i in tf.reshape(results, [max_length]).numpy() if i!=-1]) 
    output_text = ''.join([num_to_char[i] for i in resultn])
    return output_text

In [ ]:
for i, (x_batch_test, y_batch_test) in enumerate(dataset_test):
    preds = encoder(x_batch_test)
    pred_texts = greedy_predictions(preds)
    y_batch = np.array([i for i in tf.reshape(y_batch_test, [max_length]).numpy() if i != (len(characters))]) 
    label_true = ''.join([num_to_char[i] for i in y_batch])
    print(f"Actual :  {label_true},    Predicted :  {pred_texts}")   

Actual :  nine,    Predicted :  nine
Actual :  eight,    Predicted :  eight
Actual :  two,    Predicted :  tie
Actual :  nine,    Predicted :  nine
Actual :  four,    Predicted :  four
Actual :  eight,    Predicted :  eight
Actual :  five,    Predicted :  five
Actual :  zero,    Predicted :  zero
Actual :  eight,    Predicted :  eight
Actual :  seven,    Predicted :  nve
Actual :  zero,    Predicted :  zero
Actual :  eight,    Predicted :  eight
Actual :  two,    Predicted :  two
Actual :  zero,    Predicted :  zero
Actual :  one,    Predicted :  one
Actual :  zero,    Predicted :  zero
Actual :  four,    Predicted :  four
Actual :  seven,    Predicted :  seven
Actual :  nine,    Predicted :  nine
Actual :  three,    Predicted :  thre
Actual :  four,    Predicted :  four
Actual :  three,    Predicted :  three
Actual :  six,    Predicted :  six
Actual :  six,    Predicted :  six
Actual :  zero,    Predicted :  zero
Actual :  three,    Predicted :  thre
Actual :  seven,    Predicted :  s